# Effective Software Testing - Fuzzing Assignment
Both the description and the solution of this assignment will take place in this interactive jupyter notebook. Your deliverable should contain
- a `fuzzing_assignment.ipynb`, which will be the edited version of this file that will contain your solution.
- an `assignment_utils.py` that we provide you and holds helper classes for the assignment. Generally speaking, you should not edit this file, but see below for details.

The assignment *can* be solved only by filling in the cells where there is the comment "# Your code here", **but**, many solutions exist, and it is possible that some of them involve editing other cells of even the `assignment_utils.py`. The comments ("# Your code here") are **optional**, and they just indicate an exemplary solution that we will make available to you after the deadline. Feel free to edit any other cell, file, or even install new packages (in which case you must submit them in a comment at the beginning). Finally, any comments, documentation, or other remarks in natural language should take place inside this notebook, in a markdown cell.

For any question about the assignment, feel free to reach out to `konstantinos.kitsios@uzh.ch`.

Read below for more, and good luck!

# Installation
Python 3.9 is required for this assignment.
To install the required packages, create a new `pip` virtual environment and install the required libraries by running
```
pip install -r requirements.txt
```

If you encounter issues with the python version or the installation of requirements.txt, please reach out to `konstantinos.kitsios@uzh.ch`.

# Setup
The first part of the notebook sets up the environment by implementing classes/functions we talked about during the lecture.

In [ ]:
from urllib.parse import urlparse
import random
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
import pickle
import hashlib
from typing import Tuple, List, Callable, Set, Any

In [ ]:
from assignment_utils import Fuzzer, Runner, FunctionRunner, FunctionCoverageRunner, Coverage, population_coverage, Location

Remember the `urlparse()` function from the lecture? We will use this again. You might also remember that it never crashes, even with invalid urls, so we will use the corresponding `urlparse_that_crashes()`, like we did in the lecture.

In [ ]:
url = "https://uzh.ch"
urlparse(url)

In [ ]:
def urlparse_that_crashes(url: str) -> bool:
    supported_schemes = ["http", "https"]
    result = urlparse(url)
    if result.scheme not in supported_schemes:
        raise ValueError("Scheme must be one of " + repr(supported_schemes))
    if result.netloc == '':
        raise ValueError("Host must be non-empty")

    return True

Now, we will copy the code of the second live coding session we did in during the lecture. The code below implements the mutation part of a mutation-based blackbox fuzzer.

In [ ]:
def mutate(s, num_of_mutations = 1):
    mutators = [delete_random, insert_random, flip_random]

    for i in range(num_of_mutations):
        mutator_ind = random.randint(0, 2)
        selected_mutator = mutators[mutator_ind]
        s = selected_mutator(s)

    return s

In [ ]:
def delete_random(s):
    pos_to_delete = random.randint(0, len(s) - 1)
    return s[:pos_to_delete-1] + s[pos_to_delete:]

In [ ]:
def insert_random(s):
    pos_to_insert = random.randint(0, len(s)-1)
    char_to_insert = chr(random.randint(32, 64))
    return s[:pos_to_insert-1] + char_to_insert + s[pos_to_insert-1:]

In [ ]:
def flip_random(s):
    pos_to_flip  = random.randint(0, len(s)-1)
    byte_to_flip = ord(s[pos_to_flip]) # char --> byte
    bit_to_flip  = 1 << random.randint(0, 6) # 0010000 or 100000
    flipped_char = chr(byte_to_flip ^ bit_to_flip)

    return s[:pos_to_flip] + flipped_char + s[pos_to_flip+1:]

To put the function `mutate()` in use, we will implement the mutation-based blackbox fuzzer in the `MutationBlackboxFuzzer` class as following. The base class `Fuzzer` is just a placeholder that you can find in `utils.py`.

In [ ]:
class MutationBlackboxFuzzer(Fuzzer):
    """Base class for mutational fuzzing"""

    def __init__(self, seed: List[str],
                 min_mutations: int = 2,
                 max_mutations: int = 10) -> None:
        """Constructor.
        `seed` - a list of (input) strings to mutate.
        `min_mutations` - the minimum number of mutations to apply.
        `max_mutations` - the maximum number of mutations to apply.
        """
        self.seed = seed
        self.min_mutations = min_mutations
        self.max_mutations = max_mutations
        self.reset()

    def reset(self) -> None:
        """Set population to initial seed.
        To be overloaded in subclasses."""
        self.population = self.seed
        self.seed_index = 0


    def mutate(self, inp: str) -> str:
        return mutate(inp)

    def create_candidate(self) -> str:
        """Create a new candidate by mutating a population member"""
        candidate = random.choice(self.population)
        trials = random.randint(self.min_mutations, self.max_mutations)
        for i in range(trials):
            candidate = self.mutate(candidate)
        return candidate

    def fuzz(self) -> str:
        # If we did not return all the seeds, return the next seed. Only after we run 
        # out of seeds, we return a mutated version of the seed
        if self.seed_index < len(self.seed):
            # Still seeding
            self.inp = self.seed[self.seed_index]
            self.seed_index += 1
        else:
            # Mutating
            self.inp = self.create_candidate()
        return self.inp

In [ ]:
seed_input = "http://www.google.com/search?q=fuzzing"
mutation_fuzzer = MutationBlackboxFuzzer(seed=[seed_input])

Remember the `Runner` class we used in the lecture to measure the coverage achieved by a specific input. Instead of calling
```python
result = urlparse_that_crashes("https://uzh.ch")
```

you can instead do
```python
http_runner = FunctionCoverageRunner(urlparse_that_crashes) # define the function here
result, isCrash = http_runner.run("https://uzh.ch") # define the input here
```
which also gives you access to the coverage achieved:
```
list(http_runner.coverage())
```

In [ ]:
http_runner = FunctionCoverageRunner(urlparse_that_crashes) # Only initialize it once per function!
http_runner.run("https://foo.bar/")

len(list(http_runner.coverage()))

Putting it all together, we can now run our fuzzer with a single seed for 1000 trials and measure the number of covered lines  like this

In [ ]:
seed_input = "https://uzh.ch"
mutation_fuzzer = MutationBlackboxFuzzer(seed=[seed_input])

mutation_fuzzer.runs(http_runner, trials=1000)
print(mutation_fuzzer.population)

print("%d lines covered" % len(http_runner.coverage()))

# Task 01 - Investigating the Effect of Starting Seeds
In the above run, our fuzzer covers 40 lines using only one starting seed ("https://uzh.ch"). Your first task is to investigate the effect of the starting seed in fuzzer performance. We remind that fuzzer performance is usually measure by the achieved coverage of the fuzzer (higher coverage => higher probability of exposing bugs => better performance).

Compare the coverage of the `MutationBlackboxFuzzer` with one seed vs ten seeds. The choice of the ten seeds is up to you, you can either handpick ten random urls, or write a small script that randomly navigates the web and returns ten urls for more diversity.

Comparing the performance of two fuzzers (or the same fuzzer with different configurations in our case) is a nuanced topic due to the inherent randomness of fuzzing. For example, if we run the `MutationBlackboxFuzzer` with one seed and cover 40 lines, and then run it with ten seeds and cover 42 lines, can we claim that the more starting seeds the better? Probably not.
To make claims based on empirical data, we have to run statistical tests. In fuzzing, typically each fuzzer configuration is ran for 30 runs of 10,000 trials (i.e., you run the above cell 30 times), yielding an array of 30 coverage values for each fuzzer. The two arrays are then compared using the Mann-Whitney U Test with the null hypothesis that the values of the first array (i.e., coverage of the first fuzzer) are greater than those of the second array. If the p-value of the test is <0.05, we reject the null hypothesis, claiming that the first fuzzer achieves less coverage than the second one (with confidence level 1-0.05 = 95%).
You can run the Mann-Whitney U Test in python with the following code:
```python
from scipy.stats import mannwhitneyu
u_stat, p_value = mannwhitneyu(c1, c2, alternative='less')
if p_value < 0.05:
    print("Values of c1 are smaller than those in c2 with confidence level 95%")
```

In [ ]:
# Your code here

# Task 02: Investigating the Effect of More Sophisticated Mutations
The mutation function we implemented above (and in the lecture) is:
```python
def mutate(s, num_of_mutations = 1):
    mutators = [delete_random, insert_random, flip_random]

    for i in range(num_of_mutations):
        mutator_ind = random.randint(0, 2)
        selected_mutator = mutators[mutator_ind]
        s = selected_mutator(s)

    return s
```

Your task is to implement a more sophisticated mutation strategy by:
1. Extending the `mutate(s, num_of_mutations = 1)` function above to support three more mutators
    - Addition of small integers to a random byte
    - Replacing a random byte with a completely random byte value
    - Block duplucation (insert a random block of characters of `s` in a random position of `s`)
2. Extending the `mutate(self, s)` method of `MutationBlackboxFuzzer` so that with probability `p` it calls the `mutate(s, num_of_mutations = 1)` function above and with probability `1-p` mutates the input using **splicing**; Splicing (a.k.a, crossover) refers to selecting another seed from the existing population, and merging a random block of the new seed with the current seed at a random position.

Compare the performance of your new fuzzer with the previous fuzzer over 30 runs of 5000 trials each. Use a single starting seed `s=["https://www.swissinfo.ch"]`. What can you claim about the effect of more complex mutations?

Note: For more information about the exact mutations used in the most popular fuzzer, AFL, you can have a look at this blog by the AFL creator: https://lcamtuf.blogspot.com/2014/08/binary-fuzzing-strategies-what-works.html

In [ ]:
# Your code here

# Task 03: Compare With Mutation-Based Greybox Fuzzer
To improve on the blackbox fuzzer in the lecture, we implemented the greybox fuzzer below, that dynamically adds a mutated input to the seeds queue if that input covers a new path. 

Your task is to compare the performance of the (already implemented) greybox fuzzer with that of the blackbox fuzzer over 30 runs of 5000 trials each.
Use the same single starting seed as the previous experiment.

What can you claim about the performance of they greybox fuzzer?

In [ ]:
class MutationGreyboxFuzzer(MutationBlackboxFuzzer):
    """Fuzz with mutated inputs based on coverage"""

    def reset(self) -> None:
        super().reset()
        self.coverages_seen: Set[frozenset] = set()
        # Now empty; we fill this with seed in the first fuzz runs
        self.population = []

    def run(self, runner: FunctionCoverageRunner) -> Any:
        """Run function(inp) while tracking coverage.
           If we reach new coverage,
           add inp to population and its coverage to population_coverage
        """
        result, outcome = super().run(runner)
        new_coverage = frozenset(runner.coverage())
        if outcome == Runner.PASS and new_coverage not in self.coverages_seen:
            # We have new coverage
            self.population.append(self.inp)
            self.coverages_seen.add(new_coverage)

        return result

In [ ]:
# Your code here

# Task 04: Boosted Greybox Fuzzer
In page 30 of the lecture materials, we discussed about boosting the performance of greybox fuzzers by prioritizing seeds that exercise more rare paths. Your task is to implement this into a class named `BoostedMutationGreyboxFuzzer` and compare its performance against the `MutationGreyboxFuzzer`.

Specifically, you must calculate the coverage achieved for each input you feed into the program-under-test, and keep track of how frequently a specific coverage is triggered (hint: you may find the function `getPathID()` below useful for this). 

Then, when selecting the next seed to mutate, you must select with probability reversly proportional to the frequency of the coverage of each seed. As a result, seeds that exercise rarer paths will be selected more frequently.

Compare the performance of your new `BoostedMutationGreyboxFuzzer` with the base `MutationGreyboxFuzzer` in two target programs:
1. On the `urlparse_that_crashes()` program using 30 runs of 5,000 trials each and the same single starting seed as above.
2. On the `crashme()` program found below, 30 runs of using 10,000 trials and a single starting seed `s=["good"]`.

In [ ]:
def getPathID(coverage: Any) -> str:
    """Returns a unique hash for the covered statements"""
    pickled = pickle.dumps(sorted(coverage))
    return hashlib.md5(pickled).hexdigest()

In [ ]:
def crashme(s: str) -> None:
    if len(s) > 0 and s[0] == 'b':
        if len(s) > 1 and s[1] == 'a':
            if len(s) > 2 and s[2] == 'd':
                if len(s) > 3 and s[3] == '!':
                    raise Exception()

In [ ]:
class BoostedMutationGreyboxFuzzer(MutationBlackboxFuzzer):
    """Fuzz with mutated inputs based on coverage"""

    def reset(self) -> None:
        super().reset()
        self.coverages_seen: Set[frozenset] = set()
        # Now empty; we fill this with seed in the first fuzz runs
        self.population = []
        # Your code here


    def run(self, runner: FunctionCoverageRunner) -> Any:
        # Your code here

    def create_candidate(self) -> str:
        # Your code here


In [ ]:
# Your code here